In [ ]:
import pandas as pd
import numpy as np
import pickle
import sklearn

from pathlib import Path

In [ ]:
# Load data
cwd = Path.cwd()
moondf = pickle.load(open(cwd / '..' / 'raw_data' / 'moonGen_scrape_2016_final_df', 'rb'))